In [1]:
import sqlite3

## DB作成

In [2]:
path = ''
db_name = 'github_repos.db'

try:
    conn = sqlite3.connect(path + db_name)
    cur = conn.cursor()
    sql = 'CREATE TABLE repositories (id INT, name TEXT, primary_language TEXT, stars INT);'
    cur.execute(sql)
except sqlite3.Error as e:
    print('エラーが発生しました:', e)
finally:
    conn.close()

## スクレイピング

In [ ]:
import requests
from bs4 import BeautifulSoup
import time

# GitHub Google org のリポジトリ一覧
START_URL = "https://github.com/orgs/google/repositories"
WAIT_SEC = 1.0

# 取得したリポジトリ情報を保存
repositories = []

print("🚀 スクレイピング開始！\n")

page_num = 1
repo_id = 1

while True:
    target = f"{START_URL}?page={page_num}"
    print(f"▶ Fetching: {target}")

    time.sleep(WAIT_SEC)

    try:
        resp = requests.get(
            target,
            headers={"User-Agent": "MU-GitHubScraper/1.0"},
            timeout=8
        )

        if resp.status_code != 200:
            print(f"  ❌ ステータスコード {resp.status_code} (終了)")
            break

        soup = BeautifulSoup(resp.text, "html.parser")

        # リポジトリ一覧部分
        repo_list = soup.select("li.Box-row")
        if not repo_list:      # ページにリポジトリがなければ終了
            print("  ❌ リポジトリが見つからないため終了")
            break

        # ---- 各リポジトリ情報を取得 ----
        for repo in repo_list:

            # リポジトリ名
            name_tag = repo.select_one("a[href*='/google/']")
            if not name_tag:
                continue
            name = name_tag.text.strip()

            # 主要言語 → primary_language
            lang_tag = repo.select_one("span[itemprop='programmingLanguage']")
            primary_language = lang_tag.text.strip() if lang_tag else None

            # スター数
            star_tag = repo.select_one("a[href$='/stargazers']")
            if star_tag:
                stars = int(star_tag.text.strip().replace(",", ""))
            else:
                stars = 0

            # 辞書型に格納
            repositories.append({
                "id": repo_id,
                "name": name,
                "primary_language": primary_language,
                "stars": stars
            })

            print(f"  ✅ [{repo_id:03}] {name} ({primary_language}) ⭐ {stars}")

            repo_id += 1

        page_num += 1  # 次ページへ

    except Exception as err:
        print(f"  ⚠ エラー発生: {err}")
        break

# ---- 結果表示 ----
print("\n🌐 --- スクレイピング完了 ---")
for repo in repositories:
    print(f"{repo['id']:03}: {repo['name']} | {repo['primary_language']} | ⭐{repo['stars']}")

print(f"\n合計 {len(repositories)} 件のリポジトリを取得しました。")



## データ挿入

In [ ]:
path = ''
db_name = 'github_repos.db'

try:
    conn = sqlite3.connect(path + db_name)
    cur = conn.cursor()
    sql = "INSERT INTO repositories (id, name, primary_language, stars) VALUES (?, ?, ?, ?);"

    repositories = [
        (1, 'awesome-python', 'Python', 123456),
        (2, 'react', 'JavaScript', 234567),
    ]

    cur.executemany(sql, repositories)

    conn.commit()
except sqlite3.Error as e:
    print('エラーが発生しました:', e)
finally:
    conn.close()

In [3]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import time
import re

# 設定
BASE_URL = 'https://github.com/orgs/google/repositories?type=all'
DB_NAME = 'github_repos.db'
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# --- ヘルパー関数: スター数を数値に変換 ---
def parse_stars_to_int(star_text):
    if not star_text: return 0
    try:
        if 'k' in star_text:
            return int(float(star_text.replace('k', '')) * 1000)
        return int(float(star_text.replace(',', '')))
    except ValueError:
        return 0

# --- メイン処理 ---
def scrape_and_insert_directly():
    # 1. データベース接続とテーブル作成
    conn = sqlite3.connect(DB_NAME)
    cur = conn.cursor()
    
    # 既にテーブルがある場合は削除して作り直す（リセット用）
    cur.execute('DROP TABLE IF EXISTS repositories')
    
    # テーブル作成 (SQLの定義通り)
    # 順番: id, name, primary_language, stars
    cur.execute('CREATE TABLE repositories (id INT, name TEXT, primary_language TEXT, stars INT);')
    conn.commit()
    print(f"🗄️ データベース '{DB_NAME}' を作成し、接続しました。")

    page_num = 1
    total_repos = 0
    previous_page_first_repo = None
    
    try:
        while True:
            current_url = f"{BASE_URL}&page={page_num}"
            print(f"\n📄 {page_num}ページ目を処理中... ({current_url})")
            
            response = requests.get(current_url, headers=HEADERS)
            if response.status_code != 200:
                print(f"❌ エラー: ステータスコード {response.status_code}")
                break
            
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # --- 要素の特定ロジック ---
            repo_items = []
            repo_list_container = soup.find('div', class_=lambda c: c and 'ListView_module__list-view-container' in c)
            if repo_list_container:
                repo_items = repo_list_container.find_all('li')
            
            if not repo_items:
                # フォールバック
                all_li_tags = soup.find_all('li')
                for li in all_li_tags:
                    if li.find('a', class_='Title-module__anchor--GmXUE Title-module__inline--oM0P7'):
                        repo_items.append(li)

            if not repo_items:
                print("🏁 アイテムが見つかりません。終了します。")
                break
            
            # 無限ループ防止
            check_name = 'N/A'
            check_tag = repo_items[0].find('a', class_='Title-module__anchor--GmXUE Title-module__inline--oM0P7')
            if check_tag and check_tag.find('span'): check_name = check_tag.find('span').text.strip()
            
            if check_name == previous_page_first_repo:
                print("🏁 前ページと同じ内容のため終了します。")
                break
            previous_page_first_repo = check_name
            
            # --- データ抽出と整形 ---
            page_data_for_db = []
            
            for i, item in enumerate(repo_items):
                repo_id = total_repos + i + 1
                
                # 名前
                repo_name = 'N/A'
                name_tag = item.find('a', class_='Title-module__anchor--GmXUE Title-module__inline--oM0P7')
                if name_tag and name_tag.find('span'):
                    repo_name = name_tag.find('span').text.strip()
                
                # 言語
                language = 'N/A'
                lang_circle = item.find('div', class_=lambda c: c and 'LanguageCircle-module__LanguageCircleIndicator' in c)
                if lang_circle:
                    lang_span = lang_circle.find_next_sibling('span')
                    if lang_span: language = lang_span.text.strip()
                
                # スター数 (ここで数値化！)
                stars_int = 0
                star_tag = item.find('a', href=lambda h: h and '/stargazers' in h)
                if star_tag:
                    raw_text = star_tag.get_text(strip=True)
                    stars_int = parse_stars_to_int(raw_text) # ここで変換関数を使う
                
                if repo_name != 'N/A':
                    # データベースの列の順番に合わせる！
                    # (id, name, primary_language, stars)
                    page_data_for_db.append((repo_id, repo_name, language, stars_int))
            
            # --- データベースへ一括挿入 ---
            if page_data_for_db:
                sql = "INSERT INTO repositories (id, name, primary_language, stars) VALUES (?, ?, ?, ?);"
                cur.executemany(sql, page_data_for_db)
                conn.commit() # この時点で保存確定
                
                count = len(page_data_for_db)
                total_repos += count
                print(f"✅ DBに {count} 件追加 (合計: {total_repos}件)")
                
                page_num += 1
                print("💤 2秒待機...")
                time.sleep(2)
            else:
                print("⚠️ データが空でした。")
                break
                
    except Exception as e:
        print(f"❌ 予期せぬエラー: {e}")
    finally:
        conn.close()
        print("🔒 データベース接続を閉じました。処理完了！")

if __name__ == '__main__':
    scrape_and_insert_directly()

🗄️ データベース 'github_repos.db' を作成し、接続しました。

📄 1ページ目を処理中... (https://github.com/orgs/google/repositories?type=all&page=1)
✅ DBに 30 件追加 (合計: 30件)
💤 2秒待機...

📄 2ページ目を処理中... (https://github.com/orgs/google/repositories?type=all&page=2)
✅ DBに 30 件追加 (合計: 60件)
💤 2秒待機...

📄 3ページ目を処理中... (https://github.com/orgs/google/repositories?type=all&page=3)
✅ DBに 30 件追加 (合計: 90件)
💤 2秒待機...

📄 4ページ目を処理中... (https://github.com/orgs/google/repositories?type=all&page=4)
✅ DBに 30 件追加 (合計: 120件)
💤 2秒待機...

📄 5ページ目を処理中... (https://github.com/orgs/google/repositories?type=all&page=5)
✅ DBに 30 件追加 (合計: 150件)
💤 2秒待機...

📄 6ページ目を処理中... (https://github.com/orgs/google/repositories?type=all&page=6)
✅ DBに 30 件追加 (合計: 180件)
💤 2秒待機...

📄 7ページ目を処理中... (https://github.com/orgs/google/repositories?type=all&page=7)
✅ DBに 30 件追加 (合計: 210件)
💤 2秒待機...

📄 8ページ目を処理中... (https://github.com/orgs/google/repositories?type=all&page=8)
✅ DBに 30 件追加 (合計: 240件)
💤 2秒待機...

📄 9ページ目を処理中... (https://github.com/orgs/google/repositories?type=